## Importing required libraries and Loading Datasets

In [1]:
#import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import statsmodels as sm

education = pd.read_csv('education_2023.csv')
unemployment = pd.read_csv('unemployment_2023.csv')
quality_of_life = pd.read_csv('QOL.csv')
ltss_2022 = pd.read_csv('ltss_2022.csv')
ltss_expenses = pd.read_csv('ltss_expenses.csv')
us_states = pd.read_csv('us_states.csv')
gini = pd.read_csv('acs_gini.csv')
master_fips = pd.read_csv('fips_master.csv')

In [ ]:
all_fips = pd.merge(master_fips, us_states, how='left', on='state_abbr')

# Data Cleaning
see: documentation/data_model.md

## Table Normalization
### Key Values
- fips
- county
- state
- state_abbr

#### education_unemployment
#### LTSS
#### Gini_index

## Education and unemployment tables

In [102]:
# education and unemployment subset values lists (based on row values)
education_22_subset = education['Attribute'].isin(['Less than high school graduate, 2019-23', 'High school graduate (or equivalency), 2019-23', 'Some college or associate degree, 2019-23', 'Bachelor\'s degree or higher, 2019-23'])
unemployment_22_subset = unemployment['Attribute'].isin(['Civilian_labor_force_2022', 'Employed_2022', 'Unemployed_2022'])
# New dataframes for education and unemployment based on the subset values
education_2022 = education[education_22_subset]
unemployment_2022 = unemployment[unemployment_22_subset]
# defining variable for data cleaning function 
education_dtypes = {'FIPS Code': 'str', 'Area name': 'str', 'State': 'str'}
education_names = {'FIPS Code': 'fips', 'Area name':'county', 'State':'state_abbr'}
unemployment_dtypes = {'FIPS_Code': 'str', 'Area_Name': 'str', 'State': 'str'}
unemployment_names = {'FIPS_Code': 'fips', 'Area_Name':'county1', 'State':'state_abbr'}

### Cleaning Function

In [103]:
def clean_dataframe(df, data_types, names):
    df = df.astype(data_types)
    df = df.rename(columns=names)
    return df

education_2022 = clean_dataframe(education_2022, education_dtypes, education_names)
unemployment_2022 = clean_dataframe(unemployment_2022, unemployment_dtypes, unemployment_names)

### Pivoting clean dataframes
Values(rows) in 'Attribute' col are to be used as col names- recording education levels in population

In [104]:
education_2022 = education_2022.pivot_table(index=['state_abbr', 'county', 'fips'], columns='Attribute', values='Value', aggfunc='first').reset_index()
unemployment_2022 = unemployment_2022.pivot_table(index=['state_abbr', 'county1', 'fips'], columns='Attribute', values='Value', aggfunc='first').reset_index()
# renaming pivoted columns
education_2022= education_2022.rename(columns={'Less than high school graduate, 2019-23': 'no_hs_diploma', 'High school graduate (or equivalency), 2019-23':'hs_diploma', 'Some college or associate degree, 2019-23': 'some_college', 'Bachelor\'s degree or higher, 2019-23': 'bachelors_or_more'})
unemployment_2022= unemployment_2022.rename(columns={'Civilian_labor_force_2022':'labor_force', 'Employed_2022':'employed', 'Unemployed_2022':'unemployed'})

### Separating county and state function

In [105]:
# unemployment_2022: defining func. tp split state and county columns, naming new columns
def split_county(county1):
    return pd.Series(county1.split(",", 1))

unemployment_2022[['county', 'state_abbr_duplicate']] = unemployment_2022['county1'].apply(split_county)

In [106]:
# dropping duplicate columns
unemployment_2022 = unemployment_2022.drop(columns=['county1', 'state_abbr_duplicate'])

## LTSS enrollment and expenditure tables

In [107]:
# defining columns to keep and subsetting dataframes
ltss_cols= ['State', 'LTSS (total)', 'Institutional (total)', 'HCBS (total)']
ltss_population = ltss_2022[ltss_cols]
ltss_expenditures = ltss_expenses[ltss_cols]

### Cleaning invalid characters in columns to prep for datatype conversion to float

In [108]:
# removing '$' from ltss)_expenditures and changing to float
ltss_expenditures['LTSS (total)'] = ltss_expenditures['LTSS (total)'].str[1:]
ltss_expenditures['Institutional (total)'] = ltss_expenditures['Institutional (total)'].str[1:]
ltss_expenditures['HCBS (total)'] = ltss_expenditures['HCBS (total)'].str[1:]

/tmp/ipykernel_3774/778973466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltss_expenditures['LTSS (total)'] = ltss_expenditures['LTSS (total)'].str[1:]
/tmp/ipykernel_3774/778973466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltss_expenditures['Institutional (total)'] = ltss_expenditures['Institutional (total)'].str[1:]
/tmp/ipykernel_3774/778973466.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [109]:
# removing ',' from ltss)_expenditures and changing to float
ltss_expenditures['LTSS (total)'] = ltss_expenditures['LTSS (total)'].replace(',', '', regex=True).astype(float)
ltss_expenditures['Institutional (total)'] = ltss_expenditures['Institutional (total)'].replace(',', '', regex=True).astype(float)
ltss_expenditures['HCBS (total)'] = ltss_expenditures['HCBS (total)'].replace(',', '', regex=True).astype(float)

# removing ',' and changing to float - ltss_population
ltss_population['LTSS (total)'] = ltss_population['LTSS (total)'].str.replace(',', '').astype(float)
ltss_population['Institutional (total)'] = ltss_population['Institutional (total)'].str.replace(',', '').astype(float)
ltss_population['HCBS (total)'] = ltss_population['HCBS (total)'].str.replace(',', '').astype(float)

/tmp/ipykernel_3774/2481729838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltss_expenditures['LTSS (total)'] = ltss_expenditures['LTSS (total)'].replace(',', '', regex=True).astype(float)
/tmp/ipykernel_3774/2481729838.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltss_expenditures['Institutional (total)'] = ltss_expenditures['Institutional (total)'].replace(',', '', regex=True).astype(float)
/tmp/ipykernel_3774/2481729838.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

### Renaming columns

In [110]:
ltss_pop_names = {'State': 'state', 'LTSS (total)': 'ltss_total_enrollment', 'Institutional (total)': 'insitutional_enrollment', 'HCBS (total)': 'hcbs_enrollment'}
ltss_expense_names = { 'State':'state', 'LTSS (total)': 'ltss_total_expense', 'Institutional (total)': 'insitutional_expense', 'HCBS (total)': 'hcbs_expense'}

ltss_population = ltss_population.rename(columns=ltss_pop_names)
ltss_expenditures = ltss_expenditures.rename(columns=ltss_expense_names)

## GINI Index

In [111]:
gini_dytpes = { 'state': 'str' , 'county': 'str'}
gini_col_names = {'NAME': 'geography', 'B19083_001E': 'gini_index', 'state':'state_fips' , 'county':'county_fips'}

In [113]:
gini_index = clean_dataframe(gini, gini_dytpes, gini_col_names)

### split 'county' by comma to end up with cols: 'county', 'state_abbr'

In [114]:
# function to split the geography column into state and county
def split_county(geography):
    return pd.Series(geography.split(",", 1))

In [115]:
gini_index[['county', 'state']] = gini_index['geography'].apply(split_county)

In [116]:
gini_index = gini_index.drop(columns='geography')
gini_index = gini_index[~(gini_index['state'] == "Puerto Rico")]

### Normalize 'fips' by appending '00' to county fips and concatenating to state fips

In [117]:
gini_index['county_fips'] = '00' + gini_index['county_fips'].astype(str)
gini_index['fips']= gini_index['state_fips'] + gini_index['county_fips']

In [118]:
gini_index.head()

,gini_index,state_fips,county_fips,county,state,fips
0,0.4515,1,001,Autauga County,Alabama,1001
1,0.4589,1,003,Baldwin County,Alabama,1003
2,0.5061,1,005,Barbour County,Alabama,1005
3,0.4526,1,007,Bibb County,Alabama,1007
4,0.4461,1,009,Blount County,Alabama,1009


In [119]:
gini_index.to_csv('gini_index.csv', index=False)

## Quality of Life 

In [120]:
qol_cols = ['NMCNTY', 'FIPS', 'LSTATE', '2022 Population', 'Cost of Living', '2022 Median Income', 'Diversity Rank (Race)', 'Diversity Rank (Gender)']
qol_col_names = {'NMCNTY': 'county', 'FIPS': 'fips', 'LSTATE': 'state_abbr', '2022 Population': 'population', 'Cost of Living': 'cost_of_living', '2022 Median Income': 'median_income', 'Diversity Rank (Race)': 'race_diversity_rank', 'Diversity Rank (Gender)': 'gender_diversity_rank'}
qol_2022 = quality_of_life[qol_cols]

In [121]:
qol_2022 = qol_2022.rename(columns=qol_col_names)

### Clean invalid characters and change datatypes 

In [122]:
# defining variables for cleaning- removing non-numeric characters
comma_clean = ','
remove_comma = ['population', 'median_income', 'cost_of_living']
qol_floats = {'population':'float', 'median_income':'float', 'cost_of_living':'float'}


In [123]:
qol_2022[remove_comma] = qol_2022[remove_comma].replace(comma_clean, '', regex=True)
# eqivalent procvess did not work to remove '$'

In [124]:
# removing 1st character, '$'
qol_2022['median_income'] = qol_2022['median_income'].str[1:]
qol_2022['cost_of_living'] = qol_2022['cost_of_living'].str[1:]

In [125]:
# changing cleaned column data types to float
qol_2022[remove_comma]= qol_2022[remove_comma].astype(qol_floats)
qol_2022['fips'] = qol_2022['fips'].astype(str)

## Merge Tables
gini= 'state', 'county', 'fips'
qol= 'state_abbr', 'county', 'fips'
master_fips = 'fips', 'county', 'state_abbr'

In [2]:
# normalizing master_fips 'fips' column for merging
master_fips['fips'] = master_fips['fips'].astype(str)
states = pd.read_csv('us_states.csv')
states = states.drop(columns='Unnamed: 2')

In [4]:
all_fips = pd.merge(master_fips, us_states, how='outer', on='state_abbr')

In [6]:
all_fips = all_fips.drop_duplicates(subset=['fips'])
all_fips =all_fips.drop(columns= 'Unnamed: 2')
all_fips['fips'].astype(str)

0          2013
1          2016
2          2020
3          2050
4          2060
          ...  
148467    51000
151559    53000
154651    54000
157743    55000
160835    56000
Name: fips, Length: 3195, dtype: object

In [9]:
all_fips.to_csv('all_fips.csv', index=False)


### preliminary table joins on common key values

In [127]:
ltss_combined = pd.merge(ltss_population, ltss_expenditures, on='state', how='outer')
education_unemployment = pd.merge(education_2022, unemployment_2022, on=['county', 'state_abbr', 'fips'], how='outer')
quality_of_life_2022 = pd.merge(qol_2022, master_fips, on=['fips', 'county', 'state_abbr'], how='inner')


In [128]:
ltss_combined.to_csv('ltss_combined.csv', index=False)

In [129]:
# temporary table with qol, unemployment, and education
test2_df = pd.merge(education_unemployment, quality_of_life_2022, on=['county', 'state_abbr', 'fips'], how='outer')
# temporay table with values above and 'state' column (in addition to 'state_abbr')
qol_edu_unemploy = pd.merge(test2_df, states, on='state_abbr', how='left')

In [130]:
# drop duplicate 'fips' values
qol_edu_unemploy = qol_edu_unemploy.drop_duplicates(subset=['fips'])
# create new col 'county_state' in df above and gini_index
#qol_edu_unemploy['county_state'] = qol_edu_unemploy['county'] + ', ' + qol_edu_unemploy['state']
#gini_index['county_state'] = gini_index['county'] + ', ' + gini_index['state']

In [131]:
qol_edu_unemploy.to_csv('qol_edu_unemploy.csv', index=False)